<a href="https://colab.research.google.com/github/NehaSontakk/BATH-Analytics/blob/May26_Tests/BATH_file_deduplication_(Positive_Negative).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd

In [11]:
e_value_threshold = 0.000001

In [12]:
class filtering_operations:

  @staticmethod
  def e_value_filtering(df):
    return df.loc[df['E-value']<=e_value_threshold]

  @staticmethod
  def pos_neg_strand_filtering(df_filtered):
    #df should be post e_value_filteration
    df = filtering_operations.e_value_filtering(df_filtered)
    #Strand identification
    df['ali from'] = df['ali from'].astype(int)
    df['ali to'] = df['ali to'].astype(int)
    strand = []

    for index,row in df.iterrows():
      if row['ali from'] < row['ali to']:
        strand.append("+")
      elif row['ali from'] > row['ali to']:
        strand.append("-")

    df['strand'] = strand
    return df

  @staticmethod
  def get_specific_strand(df_raw,strand_info):
    df = filtering_operations.pos_neg_strand_filtering(df_raw)
    return df.loc[df['strand']==strand_info]


In [13]:
class overlap_deduplications:

  ali_to_column = ""
  ali_from_column = ""

  #METHODS FOR 100% DEDUPLICATION

  @staticmethod
  def handle_group(group):
    #Now for each element in group
    min_e_value = group['E-value'].min()
    #find elements with lowest e-values
    group_min_e = group.loc[group['E-value'] == min_e_value]
    #if group contains a single row save it
    if len(group_min_e) == 1:
      return group_min_e
    #if group contains multiple rows with same e-values check min score
    elif len(group_min_e) > 1:
      return group_min_e.iloc[0:1]

  @staticmethod
  def deduplicate_full_overlaps(df):
    #100% Overlap Dedup
    print("Number of elements in relevant strand: ",df.shape)
    #Find all the duplicates
    duplicates = df.duplicated(subset=['target name',overlap_deduplications.ali_from_column, overlap_deduplications.ali_to_column], keep=False)
    #make a duplicates df
    duplicates_df = df[duplicates]
    #make a non duplicates df
    not_duplicate_df = df[~duplicates]
    print("Number of exact duplicates or homologs: ",duplicates_df.shape)
    #Sort by E-value and score
    duplicates_df1 = duplicates_df.sort_values(by=['E-value', 'score'], ascending=[True, False])
    deduplicated_remhomologs = duplicates_df1.groupby(['target name',overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column], group_keys=False).apply(overlap_deduplications.handle_group).reset_index(drop=True)
    #rejion after deduplication
    dedup_step1 = pd.concat([not_duplicate_df,deduplicated_remhomologs]).sort_values(['target name','ali from','E-value'])
    print("Number of elements after removing exact duplicates or homologs: ",dedup_step1.shape)
    print("Elements removed: ",df.shape[0] - dedup_step1.shape[0])
    return dedup_step1

  #METHODS FOR 70% DEDUPLICATION

  @staticmethod
  def calculate_overlap(hit_a, hit_b):
    #print("Overlap calculations done on columns:",overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column)
    start_a, end_a = hit_a[overlap_deduplications.ali_from_column], hit_a[overlap_deduplications.ali_to_column]
    start_b, end_b = hit_b[overlap_deduplications.ali_from_column], hit_b[overlap_deduplications.ali_to_column]

    overlap_length = max(0, min(end_a, end_b) - max(start_a, start_b) + 1)
    #print(min(end_a, end_b))
    #print(max(start_a, start_b))
    #print("Overlap length",overlap_length)

    length_a = end_a - start_a + 1
    length_b = end_b - start_b + 1

    #print("Length",length_a,length_b)
    if length_a > 0 and length_b > 0:
        overlap_perc_a = (overlap_length / length_a) * 100
        overlap_perc_b = (overlap_length / length_b) * 100
        return overlap_perc_a, overlap_perc_b
    else:
        return 0, 0

  @staticmethod
  def calculate_winner(hit_a, hit_b):
    overlap1, overlap2 = overlap_deduplications.calculate_overlap(hit_a, hit_b)
    #print(overlap1)
    if overlap1 >= 70 or overlap2 >=70:   #PROBLEMATIC!
      if hit_a['E-value'] < hit_b['E-value']:
        return hit_a
      else:
        return hit_b
    else:
      return hit_a, hit_b

  @staticmethod
  def handle_70_overlaps(group):
    items = []
    for index, row in group.iterrows():
      row_tuple = row.to_dict()
      items.append(row_tuple)

    i = 0
    all_winners = []

    while i<len(items)-1:
      current = items[i]
      next_item = items[i+1]
      print("Begining Comparison: ",i)
      #print(current,"\n",next_item)
      result = overlap_deduplications.calculate_winner(current, next_item)

      #If result is two items append the current item to winners list since overlap of current item with next item is less than 70% so we want to keep both
      if isinstance(result,tuple):
        if result[0] not in all_winners:
          all_winners.append(result[0])
          #print("Tuple returned!")
          #print("Appending: ",result[0])
          #print("All winners currently: ",all_winners)
        #else:
        #  print("Result already in all winners, no appending!")
        i += 1
      else:
        if result == current:
          #print("Result is ",i," element and not ",i+1," popping: ",items[i+1])
          items.pop(i+1)
        else:
          i += 1

      if i >= len(items) - 1 and items[-1] not in all_winners:
        all_winners.append(items[-1])

    return pd.DataFrame(all_winners)

  #METHODS FOR LESS THAN 70% DEDUPLICATION
  @staticmethod
  def calculate_below_70_changes(hit_a, hit_b):
    overlap1, overlap2 = overlap_deduplications.calculate_overlap(hit_a, hit_b)
    #print("Overlap 1: ",overlap1)
    if (0.01 <= overlap1 < 70) and (0.01 <= overlap2 < 70):
      if hit_a['E-value'] < hit_b['E-value']:
        #print("Case A: A dominates")
        #print(hit_a)
        #print(hit_b)
        hit_b['ali from'] = hit_a['ali to'] + 1
        #print(hit_b)
        return hit_a, hit_b
      else:
        #print("Case B: B dominates")
        #print(hit_a)
        #print(hit_b)
        hit_a['ali to'] = hit_b['ali from'] - 1
        #print(hit_a)
        #print(hit_b)
        return hit_a, hit_b
    else:
      return hit_a,hit_b

  @staticmethod
  def below_70_overlap(group):
    items = []
    for index, row in group.iterrows():
      row_tuple = row.to_dict()
      items.append(row_tuple)

    for i in range(0,len(items)-1):
      element1, element2 = overlap_deduplications.calculate_below_70_changes(items[i],items[i+1])
      items[i] = element1
      items[i+1] = element2

    return pd.DataFrame(items)

  # FINAL METHOD TO CALL
  @staticmethod
  def choose_strand_operations(df):
    if df['strand'].values[0] == '-':
      print("Processing negative strand data...")
      #create the flip columns
      df['ali from flip'] = df['ali to']
      df['ali to flip'] = df['ali from']
      overlap_deduplications.ali_to_column = 'ali to flip'
      overlap_deduplications.ali_from_column = 'ali from flip'
      #100% dedup
      df = df.sort_values(by=['ali from flip','E-value'], ascending=[True, True])
      step1_dedup = overlap_deduplications.deduplicate_full_overlaps(df)
      print("Overlap calculations done on columns:",overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column)
      print("100% deduplication done.")
      #70% Overlap
      step2_dedup = step1_dedup.groupby(['target name'], group_keys=False).apply(overlap_deduplications.handle_70_overlaps).reset_index(drop=True)
      print("Overlap calculations done on columns:",overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column)
      print("70% deduplication done.")
      #Less than 70% overlap
      step3_dedup = step2_dedup.groupby(['target name'], group_keys=False).apply(overlap_deduplications.below_70_overlap).reset_index(drop=True)
      print("Less than 70% deduplication done.")
      #return something final
      return step3_dedup
    elif df['strand'].values[0] == '+':
      print("Processing positive strand data...")
      overlap_deduplications.ali_to_column = 'ali to'
      overlap_deduplications.ali_from_column = 'ali from'
      #100% dedup
      df = df.sort_values(by=['ali from','E-value'], ascending=[True, True])
      step1_dedup = overlap_deduplications.deduplicate_full_overlaps(df)
      print("Overlap calculations done on columns:",overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column)
      print("100% deduplication done.")
      #70% Overlap
      step2_dedup = step1_dedup.groupby(['target name'], group_keys=False).apply(overlap_deduplications.handle_70_overlaps).reset_index(drop=True)
      print("Overlap calculations done on columns:",overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column)
      print("70% deduplication done.")
      #Less than 70% overlap
      step3_dedup = step2_dedup.groupby(['target name'], group_keys=False).apply(overlap_deduplications.below_70_overlap).reset_index(drop=True)
      print("Less than 70% deduplication done.")
      #return something final
      return step3_dedup




In [14]:
#Do it for all BATH files we have
from glob import glob

for i in glob("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Bath_Output/*.tbl"):
  bathout = pd.read_table("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Bath_Output/DNA_Bacteria_kingdom_sprot.tbl",sep="\s+",header=None,skiprows=2,skipfooter=8)
  col_names = ['target name', 'accession', 'query name', 'accession1', 'hmm len', 'hmm from', 'hmm to', 'seq len', 'ali from', 'ali to', 'env from', 'env to', 'E-value', 'score', 'bias', 'shifts', 'stops', 'pipe', 'description of target', 'extra']
  bathout.rename(columns=dict(zip(bathout.columns, col_names)), inplace=True)
  df_neg = filtering_operations.get_specific_strand(bathout,"-")
  neg_deduped = overlap_deduplications.choose_strand_operations(df_neg)
  df_pos = filtering_operations.get_specific_strand(bathout,"+")
  pos_deduped = overlap_deduplications.choose_strand_operations(df_pos)
  bath_file_deduplication = pd.concat([pos_deduped,neg_deduped])
  bath_file_deduplication['origin'] = str(i.split("/")[-1].split(".tbl")[0])
  bath_file_deduplication.to_excel(str(i.split("/")[-1].split(".tbl")[0]+".xlsx"),index=False)

<ipython-input-14-718c2c694ba7>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  bathout = pd.read_table("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Bath_Output/DNA_Bacteria_kingdom_sprot.tbl",sep="\s+",header=None,skiprows=2,skipfooter=8)
<ipython-input-12-830c4c50e74a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ali from'] = df['ali from'].astype(int)
<ipython-input-12-830c4c50e74a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

Processing negative strand data...
Number of elements in relevant strand:  (5133, 22)
Number of exact duplicates or homologs:  (1956, 22)
Number of elements after removing exact duplicates or homologs:  (3844, 22)
Elements removed:  1289
Overlap calculations done on columns: ali from flip ali to flip
100% deduplication done.
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  1
Begining Comparison:  1
Begining Comparison:  2
Begining Comparison:  2
Begining Comparison:  2
Begining Comparison:  2
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  4
Begining Comparison:  4
Begining Comparison:  5
Begining Comparison:  5
Begining Comparison:  5
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  7
Begining Comparison:  8
B

<ipython-input-12-830c4c50e74a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ali from'] = df['ali from'].astype(int)
<ipython-input-12-830c4c50e74a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ali to'] = df['ali to'].astype(int)
<ipython-input-12-830c4c50e74a>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

Processing positive strand data...
Number of elements in relevant strand:  (4644, 20)
Number of exact duplicates or homologs:  (1541, 20)
Number of elements after removing exact duplicates or homologs:  (3653, 20)
Elements removed:  991
Overlap calculations done on columns: ali from ali to
100% deduplication done.
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  1
Begining Comparison:  1
Begining Comparison:  1
Begining Comparison:  1
Begining Comparison:  1
Begining Comparison:  1
Begining Comparison:  2
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  4
Begining Comparison:  5
Begining Comparison:  5
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Com

<ipython-input-14-718c2c694ba7>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  bathout = pd.read_table("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Bath_Output/DNA_Bacteria_kingdom_sprot.tbl",sep="\s+",header=None,skiprows=2,skipfooter=8)
<ipython-input-12-830c4c50e74a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ali from'] = df['ali from'].astype(int)
<ipython-input-12-830c4c50e74a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

...

Processing positive strand data...
Number of elements in relevant strand:  (4644, 20)
Number of exact duplicates or homologs:  (1541, 20)
Number of elements after removing exact duplicates or homologs:  (3653, 20)
Elements removed:  991
Overlap calculations done on columns: ali from ali to
100% deduplication done.
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  1
Begining Comparison:  1
Begining Comparison:  1
Begining Comparison:  1
Begining Comparison:  1
Begining Comparison:  1
Begining Comparison:  2
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  4
Begining Comparison:  5
Begining Comparison:  5
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Com

<ipython-input-14-718c2c694ba7>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  bathout = pd.read_table("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Bath_Output/DNA_Bacteria_kingdom_sprot.tbl",sep="\s+",header=None,skiprows=2,skipfooter=8)
<ipython-input-12-830c4c50e74a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ali from'] = df['ali from'].astype(int)
<ipython-input-12-830c4c50e74a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

Processing negative strand data...
Number of elements in relevant strand:  (5133, 22)
Number of exact duplicates or homologs:  (1956, 22)
Number of elements after removing exact duplicates or homologs:  (3844, 22)
Elements removed:  1289
Overlap calculations done on columns: ali from flip ali to flip
100% deduplication done.
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  1
Begining Comparison:  1
Begining Comparison:  2
Begining Comparison:  2
Begining Comparison:  2
Begining Comparison:  2
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  4
Begining Comparison:  4
Begining Comparison:  5
Begining Comparison:  5
Begining Comparison:  5
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  7
Begining Comparison:  8
B

<ipython-input-12-830c4c50e74a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ali from'] = df['ali from'].astype(int)
<ipython-input-12-830c4c50e74a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ali to'] = df['ali to'].astype(int)
<ipython-input-12-830c4c50e74a>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

Processing positive strand data...
Number of elements in relevant strand:  (4644, 20)
Number of exact duplicates or homologs:  (1541, 20)
Number of elements after removing exact duplicates or homologs:  (3653, 20)
Elements removed:  991
Overlap calculations done on columns: ali from ali to
100% deduplication done.
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  0
Begining Comparison:  1
Begining Comparison:  1
Begining Comparison:  1
Begining Comparison:  1
Begining Comparison:  1
Begining Comparison:  1
Begining Comparison:  2
Begining Comparison:  3
Begining Comparison:  3
Begining Comparison:  4
Begining Comparison:  5
Begining Comparison:  5
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Comparison:  6
Begining Com

In [15]:
list_of_dfs = []
for i in glob("/content/*.xlsx"):
  list_of_dfs.append(pd.read_excel(i))

In [16]:
bathout = pd.concat(list_of_dfs)
df_neg = filtering_operations.get_specific_strand(bathout,"-")
neg_deduped = overlap_deduplications.choose_strand_operations(df_neg)
df_pos = filtering_operations.get_specific_strand(bathout,"+")
pos_deduped = overlap_deduplications.choose_strand_operations(df_pos)
complete_bath_deduplication = pd.concat([pos_deduped,neg_deduped])

Processing negative strand data...
Number of elements in relevant strand:  (3706, 24)
Number of exact duplicates or homologs:  (3706, 24)
Number of elements after removing exact duplicates or homologs:  (467, 24)
Elements removed:  3239
Overlap calculations done on columns: ali from flip ali to flip
100% deduplication done.
Begining Comparison:  0
Begining Comparison:  1
Begining Comparison:  2
Begining Comparison:  3
Begining Comparison:  4
Begining Comparison:  5
Begining Comparison:  6
Begining Comparison:  7
Begining Comparison:  8
Begining Comparison:  9
Begining Comparison:  10
Begining Comparison:  11
Begining Comparison:  12
Begining Comparison:  13
Begining Comparison:  14
Begining Comparison:  15
Begining Comparison:  16
Begining Comparison:  17
Begining Comparison:  18
Begining Comparison:  19
Begining Comparison:  20
Begining Comparison:  21
Begining Comparison:  22
Begining Comparison:  23
Begining Comparison:  24
Begining Comparison:  25
Begining Comparison:  26
Begining 

In [17]:
complete_bath_deduplication.to_excel("bin82_BATH_deduplicated.xlsx")

In [18]:
bathout.shape

(7338, 24)